In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# ライブラリのインポート
import re
import string
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

# データセットの読み込み
train_data = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
test_data = pd.read_csv('../input/commonlitreadabilityprize/test.csv')
sub_data = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')

# train_dataのurl_legalのドメイン抽出とurl_legal&licenseのNaN埋め(blank)
train_data_url = train_data.iloc[:, 0:2]
train_data_url.iloc[:, 1] = train_data_url.iloc[:, 1].fillna('blank')
train_data_url = train_data_url.replace('https://', '', regex=True)
train_data_url = train_data_url.replace('http://', '', regex=True)

for i in range(len(train_data_url['id'])):
    if train_data_url.iloc[i, 1] != 'blank':
        end = train_data_url.iloc[i, 1].index('/')
        train_data_url.iloc[i, 1] = train_data_url.iloc[i, 1][:end]

train_data_url = train_data_url.rename(columns={'url_legal':'url_legal_domain'})
train_data_url.insert(1, 'url_legal', train_data['url_legal'])
train_data_url['license'] = train_data['license']
train_data_url['excerpt'] = train_data['excerpt']
train_data_url['target'] = train_data['target']
train_data_url['standard_error'] = train_data['standard_error']

train_data_url.iloc[:, 1] = train_data_url.iloc[:, 1].fillna('blank')
train_data_url.iloc[:, 3] = train_data_url.iloc[:, 3].fillna('blank')

# 学習データの準備 (train_data_url)
train_data_rep = []
train_data_url['taggedDocument'] = ""

for i in range(len(train_data_url['excerpt'])):
    train_data_rep.append(train_data_url['excerpt'][i].replace('\n', ' '))
    w = list(map(str, train_data_rep[i].split(" ")))
    s = TaggedDocument(words = w, tags = [train_data_url['id'][i]])
    train_data_url['taggedDocument'][i] = s

# 学習データの準備 (test_data)
test_data_rep = []
test_data['taggedDocument'] = ""

for i in range(len(test_data['excerpt'])):
    test_data_rep.append(test_data['excerpt'][i].replace('\n', ' '))
    w = list(map(str, test_data_rep[i].split(" ")))
    s = TaggedDocument(words = w, tags = [test_data['id'][i]])
    test_data['taggedDocument'][i] = s

# 単語のクリーニング (train_data_url)
for i in range(len(train_data_url['taggedDocument'])):
    for j in range(len(train_data_url['taggedDocument'][i][0])):
        for k in range(3):
            if train_data_url['taggedDocument'][i][0][j].endswith('.'):
                train_data_url['taggedDocument'][i][0][j] = train_data_url['taggedDocument'][i][0][j][:-1]
            if train_data_url['taggedDocument'][i][0][j].endswith(','):
                train_data_url['taggedDocument'][i][0][j] = train_data_url['taggedDocument'][i][0][j][:-1]
            if train_data_url['taggedDocument'][i][0][j].endswith(':'):
                train_data_url['taggedDocument'][i][0][j] = train_data_url['taggedDocument'][i][0][j][:-1]
            if train_data_url['taggedDocument'][i][0][j].endswith(';'):
                train_data_url['taggedDocument'][i][0][j] = train_data_url['taggedDocument'][i][0][j][:-1]
            if train_data_url['taggedDocument'][i][0][j].endswith(')'):
                train_data_url['taggedDocument'][i][0][j] = train_data_url['taggedDocument'][i][0][j][:-1]
            if train_data_url['taggedDocument'][i][0][j].endswith(']'):
                train_data_url['taggedDocument'][i][0][j] = train_data_url['taggedDocument'][i][0][j][:-1]
            if train_data_url['taggedDocument'][i][0][j].endswith('}'):
                train_data_url['taggedDocument'][i][0][j] = train_data_url['taggedDocument'][i][0][j][:-1]
            if train_data_url['taggedDocument'][i][0][j].endswith('>'):
                train_data_url['taggedDocument'][i][0][j] = train_data_url['taggedDocument'][i][0][j][:-1]
            if train_data_url['taggedDocument'][i][0][j].endswith('\''):
                train_data_url['taggedDocument'][i][0][j] = train_data_url['taggedDocument'][i][0][j][:-1]
            if train_data_url['taggedDocument'][i][0][j].endswith('\"'):
                train_data_url['taggedDocument'][i][0][j] = train_data_url['taggedDocument'][i][0][j][:-1]
            if train_data_url['taggedDocument'][i][0][j].endswith('!'):
                train_data_url['taggedDocument'][i][0][j] = train_data_url['taggedDocument'][i][0][j][:-1]
            if train_data_url['taggedDocument'][i][0][j].endswith('?'):
                train_data_url['taggedDocument'][i][0][j] = train_data_url['taggedDocument'][i][0][j][:-1]
            
            if train_data_url['taggedDocument'][i][0][j].startswith('.'):
                train_data_url['taggedDocument'][i][0][j] = train_data_url['taggedDocument'][i][0][j][1:]
            if train_data_url['taggedDocument'][i][0][j].startswith(','):
                train_data_url['taggedDocument'][i][0][j] = train_data_url['taggedDocument'][i][0][j][1:]
            if train_data_url['taggedDocument'][i][0][j].startswith(':'):
                train_data_url['taggedDocument'][i][0][j] = train_data_url['taggedDocument'][i][0][j][1:]
            if train_data_url['taggedDocument'][i][0][j].startswith(';'):
                train_data_url['taggedDocument'][i][0][j] = train_data_url['taggedDocument'][i][0][j][1:]
            if train_data_url['taggedDocument'][i][0][j].startswith('('):
                train_data_url['taggedDocument'][i][0][j] = train_data_url['taggedDocument'][i][0][j][1:]
            if train_data_url['taggedDocument'][i][0][j].startswith('['):
                train_data_url['taggedDocument'][i][0][j] = train_data_url['taggedDocument'][i][0][j][1:]
            if train_data_url['taggedDocument'][i][0][j].startswith('{'):
                train_data_url['taggedDocument'][i][0][j] = train_data_url['taggedDocument'][i][0][j][1:]
            if train_data_url['taggedDocument'][i][0][j].startswith('<'):
                train_data_url['taggedDocument'][i][0][j] = train_data_url['taggedDocument'][i][0][j][1:]
            if train_data_url['taggedDocument'][i][0][j].startswith('\''):
                train_data_url['taggedDocument'][i][0][j] = train_data_url['taggedDocument'][i][0][j][1:]
            if train_data_url['taggedDocument'][i][0][j].startswith('\"'):
                train_data_url['taggedDocument'][i][0][j] = train_data_url['taggedDocument'][i][0][j][1:]
            if train_data_url['taggedDocument'][i][0][j].startswith('!'):
                train_data_url['taggedDocument'][i][0][j] = train_data_url['taggedDocument'][i][0][j][1:]
            if train_data_url['taggedDocument'][i][0][j].startswith('?'):
                train_data_url['taggedDocument'][i][0][j] = train_data_url['taggedDocument'][i][0][j][1:]
        
        train_data_url['taggedDocument'][i][0][j] = train_data_url['taggedDocument'][i][0][j].lower()

# 単語のクリーニング (test_data)
for i in range(len(test_data['taggedDocument'])):
    for j in range(len(test_data['taggedDocument'][i][0])):
        for k in range(3):
            if test_data['taggedDocument'][i][0][j].endswith('.'):
                test_data['taggedDocument'][i][0][j] = test_data['taggedDocument'][i][0][j][:-1]
            if test_data['taggedDocument'][i][0][j].endswith(','):
                test_data['taggedDocument'][i][0][j] = test_data['taggedDocument'][i][0][j][:-1]
            if test_data['taggedDocument'][i][0][j].endswith(':'):
                test_data['taggedDocument'][i][0][j] = test_data['taggedDocument'][i][0][j][:-1]
            if test_data['taggedDocument'][i][0][j].endswith(';'):
                test_data['taggedDocument'][i][0][j] = test_data['taggedDocument'][i][0][j][:-1]
            if test_data['taggedDocument'][i][0][j].endswith(')'):
                test_data['taggedDocument'][i][0][j] = test_data['taggedDocument'][i][0][j][:-1]
            if test_data['taggedDocument'][i][0][j].endswith(']'):
                test_data['taggedDocument'][i][0][j] = test_data['taggedDocument'][i][0][j][:-1]
            if test_data['taggedDocument'][i][0][j].endswith('}'):
                test_data['taggedDocument'][i][0][j] = test_data['taggedDocument'][i][0][j][:-1]
            if test_data['taggedDocument'][i][0][j].endswith('>'):
                test_data['taggedDocument'][i][0][j] = test_data['taggedDocument'][i][0][j][:-1]
            if test_data['taggedDocument'][i][0][j].endswith('\''):
                test_data['taggedDocument'][i][0][j] = test_data['taggedDocument'][i][0][j][:-1]
            if test_data['taggedDocument'][i][0][j].endswith('\"'):
                test_data['taggedDocument'][i][0][j] = test_data['taggedDocument'][i][0][j][:-1]
            if test_data['taggedDocument'][i][0][j].endswith('!'):
                test_data['taggedDocument'][i][0][j] = test_data['taggedDocument'][i][0][j][:-1]
            if test_data['taggedDocument'][i][0][j].endswith('?'):
                test_data['taggedDocument'][i][0][j] = test_data['taggedDocument'][i][0][j][:-1]
            
            if test_data['taggedDocument'][i][0][j].startswith('.'):
                test_data['taggedDocument'][i][0][j] = test_data['taggedDocument'][i][0][j][1:]
            if test_data['taggedDocument'][i][0][j].startswith(','):
                test_data['taggedDocument'][i][0][j] = test_data['taggedDocument'][i][0][j][1:]
            if test_data['taggedDocument'][i][0][j].startswith(':'):
                test_data['taggedDocument'][i][0][j] = test_data['taggedDocument'][i][0][j][1:]
            if test_data['taggedDocument'][i][0][j].startswith(';'):
                test_data['taggedDocument'][i][0][j] = test_data['taggedDocument'][i][0][j][1:]
            if test_data['taggedDocument'][i][0][j].startswith('('):
                test_data['taggedDocument'][i][0][j] = test_data['taggedDocument'][i][0][j][1:]
            if test_data['taggedDocument'][i][0][j].startswith('['):
                test_data['taggedDocument'][i][0][j] = test_data['taggedDocument'][i][0][j][1:]
            if test_data['taggedDocument'][i][0][j].startswith('{'):
                test_data['taggedDocument'][i][0][j] = test_data['taggedDocument'][i][0][j][1:]
            if test_data['taggedDocument'][i][0][j].startswith('<'):
                test_data['taggedDocument'][i][0][j] = test_data['taggedDocument'][i][0][j][1:]
            if test_data['taggedDocument'][i][0][j].startswith('\''):
                test_data['taggedDocument'][i][0][j] = test_data['taggedDocument'][i][0][j][1:]
            if test_data['taggedDocument'][i][0][j].startswith('\"'):
                test_data['taggedDocument'][i][0][j] = test_data['taggedDocument'][i][0][j][1:]
            if test_data['taggedDocument'][i][0][j].startswith('!'):
                test_data['taggedDocument'][i][0][j] = test_data['taggedDocument'][i][0][j][1:]
            if test_data['taggedDocument'][i][0][j].startswith('?'):
                test_data['taggedDocument'][i][0][j] = test_data['taggedDocument'][i][0][j][1:]
        
        test_data['taggedDocument'][i][0][j] = test_data['taggedDocument'][i][0][j].lower()

# train dataとtest dataのマージ
train_docs = train_data_url['taggedDocument'].values.tolist() + test_data['taggedDocument'].values.tolist()

# 学習の実行
model = Doc2Vec(
    documents = train_docs,
    dm = 1,
    vector_size = 300,
    window = 15,
    alpha = 0.0025,
    workers = 4,
    epochs = 300,
    hs = 0,
    negative = 5,
    min_count = 1)

# Doc2Vecの学習済みモデルからベクトルを特徴量として抽出
train_docvecs_df = pd.DataFrame()
test_docvecs_df = pd.DataFrame()

for did in train_data_url["id"]:
    train_docvecs_df[did] = model.dv[did]

for did in test_data["id"]:
    test_docvecs_df[did] = model.dv[did]

train_docvecs_df = train_docvecs_df.T
train_docvecs_df = train_docvecs_df.rename_axis('id').reset_index()

test_docvecs_df = test_docvecs_df.T
test_docvecs_df = test_docvecs_df.rename_axis('id').reset_index()

# LightGBMでの訓練データと評価データ
train_X, val_X, train_y, val_y = train_test_split(train_docvecs_df.drop('id', axis=1), train_data_url['target'], test_size = 0.33, random_state=42)
lgb_train = lgb.Dataset(train_X.values, train_y.values)
lgb_eval = lgb.Dataset(val_X.values, val_y.values, reference=lgb_train)

# LightGBMで学習実行
params = {
    # 回帰問題
    'objective': 'regression',
    # RMSEで評価
    'metric': 'rmse',
    'max_depth': 3
}

lgbModel = lgb.train(params, lgb_train, valid_sets = lgb_eval,
                     verbose_eval = 100,  # 50イテレーション毎に学習結果出力
                     num_boost_round = 1000,  # 最大イテレーション回数指定
                     early_stopping_rounds = 500
                    )

# 評価用データで予測
from sklearn import metrics
y_pred = lgbModel.predict(val_X.values, num_iteration = lgbModel.best_iteration)
rmse = np.sqrt(metrics.mean_squared_error(val_y.values, y_pred))

# testデータで予測
predicted = lgbModel.predict(test_docvecs_df.drop('id', axis = 1).values, num_iteration = lgbModel.best_iteration)

# 提出用データの作成
sub_data['target'] = predicted
sub_data.to_csv('submission.csv', index = False)
